In [1]:
import importlib
import sys
import torch
import pickle
import os

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')

from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [2]:
#load model
file_path_model = '../../../training_variational_dropout/BPIC17/BPIC_2017_full_grad_norm_proactive_conf_check.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../../../../risk_controlled_proactive_conformance_checking_dev/data/encoded_data/BPIC17/BPIC-17_all_normal_3_test.pkl'
bpic_17_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('concept:name', 28, {'A_Accepted': 1, 'A_Cancelled': 2, 'A_Complete': 3, 'A_Concept': 4, 'A_Create Application': 5, 'A_Denied': 6, 'A_Incomplete': 7, 'A_Pending': 8, 'A_Submitted': 9, 'A_Validating': 10, 'EOS': 11, 'O_Accepted': 12, 'O_Cancelled': 13, 'O_Create Offer': 14, 'O_Created': 15, 'O_Refused': 16, 'O_Returned': 17, 'O_Sent (mail and online)': 18, 'O_Sent (online only)': 19, 'W_Assess potential fraud': 20, 'W_Call after offers': 21, 'W_Call incomplete files': 22, 'W_Complete application': 23, 'W_Handle leads': 24, 'W_Personal Loan collection': 25, 'W_Shortened completion ': 26, 'W_Validate application': 27}), ('Action', 7, {'Created': 1, 'Deleted': 2, 'EOS': 3, 'Obtained': 4, 'Released': 5, 'statechange': 6}), ('org:resource', 150, {'EOS': 1, 'User_1': 2, 'User_10': 3, 'User_100': 4, 'User_101': 5, 'User_102': 6, 'User_103': 7, 'User_104': 8, 'User_105': 9, 'User_106': 10, 'User_107': 11, 'User_108': 12, 'User_109': 13, 'User_11': 14, 'User_110': 15, 'U

In [3]:
import evaluation.probabilistic_evaluation
importlib.reload(evaluation.probabilistic_evaluation)
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation

#device = torch.device("cuda")  # Use the first available GPU
#model = model.to(device)

new_eval = ProbabilisticEvaluation(model, 
                                   bpic_17_test_dataset,
                                   # Change here the number of cores for workers:
                                   num_processes=128,
                                   growing_num_values = ['case_elapsed_time'],
                                   # Change here number of samples:
                                   samples_per_case = 100,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   all_cat= ['concept:name'],
                                   all_num= ['case_elapsed_time', 'event_elapsed_time'])

In [4]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")

output_dir = '../../../../../../../data/BPIC17/proact_conf_check/eval_test_set/'

save_every = 500

results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=False, include_model_states=False)):
    # print(case_name, prefix_len)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    # print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)

  0%|          | 0/4726 [00:00<?, ?it/s]

Saved 500 results to ../../../../../../../data/BPIC17/proact_conf_check/eval_test_set/results_part_500.pkl


Saved 500 results to ../../../../../../../data/BPIC17/proact_conf_check/eval_test_set/results_part_1000.pkl


Saved 500 results to ../../../../../../../data/BPIC17/proact_conf_check/eval_test_set/results_part_1500.pkl


Saved 500 results to ../../../../../../../data/BPIC17/proact_conf_check/eval_test_set/results_part_2000.pkl


...

Saved 500 results to ../../../../../../../data/BPIC17/proact_conf_check/eval_test_set/results_part_172000.pkl


Saved 500 results to ../../../../../../../data/BPIC17/proact_conf_check/eval_test_set/results_part_172500.pkl


Saved 500 results to ../../../../../../../data/BPIC17/proact_conf_check/eval_test_set/results_part_173000.pkl


Saved 500 results to ../../../../../../../data/BPIC17/proact_conf_check/eval_test_set/results_part_173500.pkl


Saved 114 results to ../../../../../../../data/BPIC17/proact_conf_check/eval_test_set/results_part_173614.pkl
